# Training Material: ALOS-2 Layer Stacking

## I. Introduction to ALOS-2 Data (9.1.1)

* **Overview of ALOS-2 Satellite:** ALOS-2, or Advanced Land Observing Satellite-2 ("Daichi-2" in Japanese), is an Earth observation satellite developed by the Japan Aerospace Exploration Agency (JAXA). Its primary mission is to acquire high-resolution land observation data for various applications.  It carries two main sensors:
    * **PALSAR-2:**  A Phased Array type L-band Synthetic Aperture Radar (SAR). SAR transmits microwave signals and measures the backscattered energy to create images of the Earth's surface, regardless of weather or daylight conditions. L-band is particularly useful for penetrating vegetation and observing ground features.
    * **CIRC:** Compact InfraRed Camera, a multi-spectral optical sensor capturing data in the visible and near-infrared wavelengths.  It's used for land cover classification, vegetation monitoring, and disaster assessment.

For more detailed information, refer to the [ALOS Data Users Handbook](https://www.eorc.jaxa.jp/ALOS/en/doc/fdata/ALOS_HB_RevC_EN.pdf) and the [EO Portal ALOS-2 page](https://www.eoportal.org/satellite-missions/alos-2).

* **Key Applications:** ALOS-2 data is utilized in a wide range of applications, including:
    * **Disaster Monitoring:** Assessing damage from earthquakes, floods, and other natural disasters.
    * **Land and Agriculture Studies:** Monitoring crop growth, land use changes, and agricultural practices.
    * **Natural Resource Exploration:** Mapping geological features and identifying potential resources.
    * **Forest and Plantation Change Detection:**  ALOS-2's L-band SAR is particularly effective at penetrating forest canopies, making it ideal for detecting changes in forest cover, deforestation, and plantation development.  This capability is crucial for sustainable forest management and monitoring illegal logging activities.  See [Generating Large-Scale High-Quality SAR Mosaic Datasets](https://www.researchgate.net/publication/224183347_Generating_Large-Scale_High-Quality_SAR_Mosaic_Datasets_Application_to_PALSAR_Data_for_Global_Monitoring) for more information on large-scale SAR mosaics.

## II. Reading and Visualizing ALOS-2 Data (9.1.2)

* **Python Basics for EO Data:**

    * **File I/O:** Python provides built-in functions for file input and output.  The `open()` function is used to open files, with modes like 'r' for reading, 'w' for writing, and 'a' for appending.  See the [Python documentation on file I/O](https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files).

In [ ]:
with open('example.txt', 'w') as f:
    f.write('This is an example file.')

with open('example.txt', 'r') as f:
    content = f.read()
    print(content)

    * **Data Structures:** Python offers several data structures: lists (mutable, ordered sequences), dictionaries (mutable, unordered key-value pairs), tuples (immutable, ordered sequences), and sets (mutable, unordered collections of unique elements).  Mutability refers to whether the structure can be changed after creation.  Refer to the [Python documentation on data structures](https://docs.python.org/3/tutorial/datastructures.html).

In [ ]:
my_list = [1, 2, 3]
my_dict = {'a': 1, 'b': 2}
my_tuple = (1, 2, 3)
my_set = {1, 2, 3}

    * **Functions:** Functions are blocks of reusable code. They improve code organization and readability.  They are defined using the `def` keyword.

In [ ]:
def add(x, y):
    return x + y

result = add(5, 3)
print(result)

    * **Basic Data Manipulation (NumPy):** NumPy is essential for numerical operations in Python.  It provides powerful arrays and functions for data manipulation.  Slicing, indexing, and other operations are fundamental.

In [ ]:
import numpy as np

arr = np.array([1, 2, 3, 4, 5])
print(arr[1:4])  # Slicing
print(arr[2])   # Indexing

 * **Loading ALOS-2 Data:**

    * **Data Format:** ALOS-2 data is commonly provided in GeoTIFF format, a widely used format for georeferenced raster data. It contains both the image data and geospatial metadata (coordinates, projection).

    * **Rasterio Library:** `rasterio` is a Python library for reading and writing geospatial raster data. It simplifies working with GeoTIFFs and other formats.

In [ ]:
import rasterio

# Replace with your actual file path
image_path = 'example.tif' 

with rasterio.open(image_path) as src:
    band1 = src.read(1)  # Read the first band
    transform = src.transform  # Access geospatial transform
    crs = src.crs  # Access coordinate reference system
    print(band1.shape)
    print(transform)
    print(crs)

* **Visualizing ALOS-2 Data:**

    * **Matplotlib Library:** `matplotlib` is a powerful plotting library in Python.  It's used to create various types of visualizations, including raster data display.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example data (replace with your ALOS-2 data)
data = np.random.rand(100, 100)

plt.figure(figsize=(8, 6))
plt.imshow(data, cmap='gray')
plt.colorbar(label='Pixel Value')
plt.title('ALOS-2 Image Example')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.show()

## III. Preprocessing ALOS-2 Data for Layer Stacking

* **Importance of Preprocessing:** Preprocessing steps like calibration and filtering are crucial for preparing ALOS-2 data for layer stacking. Calibration ensures consistent data values, while filtering reduces noise and enhances image quality.

* **Radiometric Calibration:**

    * **Purpose:** Radiometric calibration converts the digital numbers in the ALOS-2 data to physical units, typically sigma-naught (σ0), which represents the radar backscatter coefficient.  This is often expressed in decibels (dB).

    * **Implementation:** The calibration formula is applied using NumPy.

In [ ]:
import numpy as np

# Example ALOS-2 data (replace with your data)
dn = np.random.rand(100, 100) * 10000

# Calibration formula (adjust constants as needed)
sigma0 = (20 * np.log10(dn)) - 83
print(sigma0)

* **Speckle Filtering:**

    * **Purpose:** Speckle noise is inherent in SAR imagery, appearing as a grainy texture.  Filtering reduces this noise, improving image interpretation.

    * **Lee Filter:** The Lee filter is a commonly used adaptive filter for speckle reduction. It preserves edges while smoothing homogeneous areas.

    * **Implementation:** The Lee filter implementation involves calculating local statistics (mean, variance) and applying weights based on these statistics.

In [ ]:
from scipy.ndimage import uniform_filter
import numpy as np

def lee_filter(img, size=7):
    img_mean = uniform_filter(img, (size, size))
    img_sq_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sq_mean - img_mean**2

    overall_variance = np.var(img)
    img_noise_variance = np.maximum(img_variance - overall_variance, 0)

    weights = img_noise_variance / (img_noise_variance + img_variance)
    filtered_img = img_mean + weights * (img - img_mean)
    return filtered_img

# Example usage (replace with your data)
img = np.random.rand(100, 100)
filtered_img = lee_filter(img)
print(filtered_img)

* **Clipping to AOI (Optional):**

    * **Purpose:** Clipping restricts processing to a specific Area of Interest (AOI), reducing processing time and focusing on the relevant region.

    * **Geopandas:** `geopandas` is a library for working with vector data (points, lines, polygons) in Python. It's used to read shapefiles, which commonly define AOIs.

    * **Rasterio Masking:** `rasterio.mask` is used to clip a raster based on a shapefile.

In [ ]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd

# Replace with your actual file paths
image_path = 'example.tif'
shapefile_path = 'aoi.shp'

# Read the shapefile
aoi = gpd.read_file(shapefile_path)

# Clip the raster
with rasterio.open(image_path) as src:
    out_image, out_transform = mask(src, aoi.geometry, crop=True)
    out_meta = src.meta.copy()

    # Update metadata
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })

    # Write the clipped raster (optional)
    # with rasterio.open('clipped.tif', 'w', **out_meta) as dest:
    #     dest.write(out_image)

print(out_image.shape)

* **Visualization of Preprocessing Steps:** Visualizing the clipped, calibrated, and filtered images side-by-side helps assess the impact of each preprocessing step.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example data (replace with your preprocessed data)
clipped_image = np.random.rand(50, 50)
calibrated_image = clipped_image * 2
filtered_image = calibrated_image + 0.1

fig, axs = plt.subplots(1, 3, figsize=(12, 4))
axs[0].imshow(clipped_image, cmap='gray')
axs[0].set_title('Clipped')
axs[1].imshow(calibrated_image, cmap='gray')
axs[1].set_title('Calibrated')
axs[2].imshow(filtered_image, cmap='gray')
axs[2].set_title('Filtered')
plt.show()

## IV. Layer Stacking

* **Concept of Layer Stacking:** Layer stacking combines multiple raster layers into a single multi-band raster. This is useful for integrating information from different dates or different sensor bands.

* **NumPy Stacking:** `np.stack` is used to stack NumPy arrays along a specified axis.

In [ ]:
import numpy as np

# Example data (replace with your preprocessed ALOS-2 data)
band1 = np.random.rand(50, 50)
band2 = np.random.rand(50, 50)
band3 = np.random.rand(50, 50)

stacked_bands = np.stack([band1, band2, band3], axis=0)  # Stack along the first axis
print(stacked_bands.shape)

* **Updating Metadata:** When stacking layers, the metadata needs to be updated to reflect the new number of bands (`count`) and the data type (`dtype`).  This ensures the stacked raster is correctly interpreted by GIS software.

In [ ]:
meta = {'count': 1, 'dtype': 'uint16'}
meta['count'] = 3
meta['dtype'] = 'float32'
print(meta)

* **Writing the Stacked Raster:** The stacked array is written to a new GeoTIFF file using `rasterio`.

In [ ]:
import rasterio
import numpy as np

# Example data and metadata (replace with your data)
stacked_data = np.random.rand(3, 50, 50)
meta = {'driver': 'GTiff', 'height': 50, 'width': 50, 'count': 3, 'dtype': 'float32', 'crs': 'EPSG:4326', 'transform': rasterio.Affine(1.0, 0.0, 0.0, 0.0, -1.0, 0.0)}

output_path = 'stacked.tif'
with rasterio.open(output_path, 'w', **meta) as dst:
    dst.write(stacked_data)

## V. Time-Series Analysis and Interpretation (9.2 & 9.3)

* **Interpreting ALOS-2 Data:**  Interpretation of ALOS-2 imagery involves analyzing backscatter patterns, texture, and changes over time.  Smooth areas typically correspond to water or homogeneous surfaces, while rough textures indicate vegetation or urban areas.  Brightness in SAR imagery relates to the strength of the radar return.

* **Comparison with Optical Imagery:** Comparing ALOS-2 SAR data with optical imagery (e.g., Landsat, Sentinel-2) provides complementary information. Optical imagery offers spectral information about surface reflectance, while SAR provides information about surface structure and scattering properties.

* **Time-Series Observation Preparation:**  Time-series analysis requires preparing data from multiple time steps.  This involves the preprocessing steps described in Section III (calibration, filtering, and optionally clipping).

* **Visualization and Interpretation of Change:**  QGIS, a powerful open-source GIS software, offers tools for visualizing and interpreting change over time.

## VI. Hands-on Exercises

Exercises will be provided separately with sample data and expected outputs.

## VII. Additional Resources

* **Rasterio:** [https://rasterio.readthedocs.io/en/latest/](https://rasterio.readthedocs.io/en/latest/)
* **Geopandas:** [https://geopandas.org/en/stable/](https://geopandas.org/en/stable/)
* **Matplotlib:** [https://matplotlib.org/](https://matplotlib.org/)
* **QGIS:** [https://qgis.org/en/site/](https://qgis.org/en/site/)